# CalFinder

Imagine you have hundred(s) of calibration curves e.g. from a targeted metabolomics experiment and you have to manually exclude calibration levels to obtain acceptable R_squared and Limit of Quantification (LOQ) values. The following code helps to automate the exclusion process of calibration levels e.g. from Skyline calibration data, so that linearity and LOQ are optimised. 

In [Skyline](https://www.skyline.ms/project/home/begin.view) the LOQ values are calculated based on external standard replicates. The LOQ value is the lowest standard concentration in the calibration curve that satisfies the 'Maximum LOQ bias' criteria, which is the maximum allowed difference (in %) between the actual analyte concentration and the calculated value from the calibration curve. The other criteria 'Maximum LOQ CV' - which is not included here - is the maximum allowed %CV of standard replicates.



## 1. Inspect calibration data

In [65]:
# Import libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [66]:
# Read calibration data from csv
data = pd.read_csv("example_data/input.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Molecule                  64 non-null     object 
 1   Replicate                 64 non-null     object 
 2   Sample Type               64 non-null     object 
 3   Analyte Concentration     56 non-null     float64
 4   Accuracy                  53 non-null     object 
 5   Exclude From Calibration  64 non-null     bool   
 6   Slope                     64 non-null     float64
 7   Intercept                 64 non-null     float64
 8   R Squared                 64 non-null     float64
 9   Limit Of Detection        16 non-null     float64
 10  Limit Of Quantification   32 non-null     float64
 11  Total Area                55 non-null     float64
dtypes: bool(1), float64(7), object(4)
memory usage: 5.7+ KB


In [67]:
data.head(10)

,Molecule,Replicate,Sample Type,Analyte Concentration,Accuracy,Exclude From Calibration,Slope,Intercept,R Squared,Limit Of Detection,Limit Of Quantification,Total Area
0,Choline,240430_0uM_r01,Blank,NaN,NaN,False,4829400.0,493390.0,0.66012,NaN,NaN,NaN
1,Choline,240430_0uM_r02,Blank,NaN,NaN,False,4829400.0,493390.0,0.66012,NaN,NaN,NaN
2,Choline,240430_S5_0.005uM_r01,Standard,0.005,-1548.3%,False,4829400.0,493390.0,0.66012,NaN,NaN,119525.0
3,Choline,240430_S5_0.01uM_r01,Standard,0.010,-531.4%,False,4829400.0,493390.0,0.66012,NaN,NaN,236744.0
4,Choline,240430_S5_0.025uM_r01,Standard,0.025,130%,False,4829400.0,493390.0,0.66012,NaN,NaN,650389.0
5,Choline,240430_S5_0.05uM_r01,Standard,0.050,324.5%,False,4829400.0,493390.0,0.66012,NaN,NaN,1276978.0
6,Choline,240430_S5_0.1uM_r01,Standard,0.100,432.8%,False,4829400.0,493390.0,0.66012,NaN,NaN,2583479.0
7,Choline,240430_S5_0.25uM_r01,Standard,0.250,497.4%,False,4829400.0,493390.0,0.66012,NaN,NaN,6498801.0
8,Choline,240430_S5_0.5uM_r01,Standard,0.500,500.3%,False,4829400.0,493390.0,0.66012,NaN,NaN,12573037.0
9,Choline,240430_S5_1uM_r01,Standard,1.000,455.9%,False,4829400.0,493390.0,0.66012,NaN,NaN,22509414.0


In [68]:
# Adjust column names
data.columns = data.columns.str.lower().str.replace(' ', '_')

# Filter for 'sample_type' = standard and reindex
data = data[data['sample_type'].str.lower() == 'standard'].reset_index(drop=True)

# Optional, filter for molecule for testing
# data = data[data['molecule']=='Choline'].reset_index(drop=True)

# Filter for required columns
data = data[['molecule', 'analyte_concentration', 'total_area']]

# Remove rows where 'total_area' is NaN 
data = data.dropna(subset=['total_area'])

In [69]:
# Check operations
print(data.shape)
print(data['molecule'].unique())
data.head()

(53, 3)
['Choline' 'Lac' 'Glu_pos' 'Glu_neg']


,molecule,analyte_concentration,total_area
0,Choline,0.005,119525.0
1,Choline,0.010,236744.0
2,Choline,0.025,650389.0
3,Choline,0.050,1276978.0
4,Choline,0.100,2583479.0


## 2. Apply CalFinder functions to calibration data

In [70]:
# CalFinder functions
import numpy as np
import pandas as pd
import statsmodels.api as sm

def load_data(file_path):
    """Load calibration curve data from a CSV file."""
    data = pd.read_csv(file_path)
    return data

def clean_data(data):
    """Clean input data."""
    # Adjust column names
    data.columns = data.columns.str.lower().str.replace(' ', '_')
    
    # Filter for 'sample_type' = standard
    if 'sample_type' in data.columns:
        data = data[data['sample_type'].str.lower() == 'standard']
    
    # Optional, filter for molecule for testing
    # if 'molecule' in data.columns:
    #     data = data[data['molecule'].str.lower() == 'choline']
    
    # Remove rows where 'total_area' is NaN
    data = data.dropna(subset=['total_area'])

    # Filter for required columns
    filtered_data = data[['molecule', 'analyte_concentration', 'total_area']]
    return filtered_data

def weighted_linear_regression(x, y, weights):
    """Perform a weighted linear regression."""
    x = sm.add_constant(x)  # Add intercept term
    model = sm.WLS(y, x, weights=weights)
    results = model.fit()
    return results

def calculate_r_squared(y_true, y_pred):
    """Calculate the R-squared value."""
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return np.round(1 - (ss_res / ss_tot), 5)

def calculate_accuracy(y_true, y_pred):
    """Calculate the accuracy in percentage for each level."""
    return np.round(100 * (y_pred / y_true), 1)

def determine_loq(accuracies, analyte_concentrations, max_loq_bias = 30):
    """Determine the LOQ as the analyte concentration one level higher than the level with a ±30% accuracy."""
    index_result = []
    for index, accuracy in enumerate(accuracies):
        # Check if the accuracy is outside the acceptable range
        if accuracy < (100 - max_loq_bias) or accuracy > (100 + max_loq_bias):
            index_result.append(index)
    
    if not index_result:
        # Set to the first index if no values meet the condition
        max_value = -1
    elif len(analyte_concentrations) == (max(index_result) + 1):
        max_value = max(index_result)
    else:
        max_value = max(index_result)
        
    max_value = min(max_value + 1, len(analyte_concentrations) - 1)
    # print(max_value)
    return analyte_concentrations.iloc[max_value]

def process_data(file_path):
    data = load_data(file_path)
    cleaned_data = clean_data(data)
    
    molecules = cleaned_data['molecule'].unique()
    all_results = []

    for molecule in molecules:
        molecule_data = cleaned_data[cleaned_data['molecule'] == molecule]
        molecule_data = molecule_data.sort_values(by='analyte_concentration')
        x = molecule_data['analyte_concentration']
        y = molecule_data['total_area']
        weights = 1 / x
        # print(weights)
        
        for n_levels in range(len(x), 4, -1):
            current_x = x.iloc[:n_levels]
            current_y = y.iloc[:n_levels]
            current_weights = weights.iloc[:n_levels]
            # print(current_weights)
            
            model = weighted_linear_regression(current_x, current_y, current_weights)
            y_pred = model.predict(sm.add_constant(current_x))
            r_squared = calculate_r_squared(current_y, y_pred)
            accuracies = calculate_accuracy(current_y, y_pred)
            
            # Debug: print accuracies
            # print(f'accuracies: {accuracies}')
            
            loq = determine_loq(accuracies, current_x)
            # print(accuracies)
            # print(current_x)
            # for index, value in enumerate(current_x):
            #     print(index, value)
            # break
        
            result = {
                'molecule': molecule,
                'num_levels': n_levels,
                'levels': list(current_x),
                'slope': model.params[1],
                'intercept': model.params[0],
                'r_squared': r_squared,
                'upper_level': max(list(current_x)),
                'loq': loq,
                'accuracies': list(accuracies)
                }
            all_results.append(result)

    results_df = pd.DataFrame(all_results)
    
    results_df.to_csv('01_all_results.csv', index=False)
    return results_df

In [71]:
# Process calibration data
df = process_data('example_data/input.csv')

## 3. Filter CalFinder results for lowest LOQ and r_squared > 0.99

In [72]:
# Load results
df = pd.read_csv('01_all_results.csv')
df.head(15)

,molecule,num_levels,levels,slope,intercept,r_squared,upper_level,loq,accuracies
0,Choline,14,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0...",4.829358e+06,493390.784435,0.66012,100.00,100.000,"[433.0, 228.8, 94.4, 57.5, 37.8, 26.2, 23.1, 2..."
1,Choline,13,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0...",6.761460e+06,431709.643864,0.72968,50.00,50.000,"[389.5, 210.9, 92.4, 60.3, 42.9, 32.7, 30.3, 3..."
2,Choline,12,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0...",9.365032e+06,355077.041473,0.76538,25.00,25.000,"[336.2, 189.5, 90.6, 64.5, 50.0, 41.5, 40.1, 4..."
3,Choline,11,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0...",1.325265e+07,249736.456663,0.89351,10.00,10.000,"[264.4, 161.5, 89.3, 71.4, 61.0, 54.8, 54.7, 6..."
4,Choline,10,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0...",1.664912e+07,168062.350121,0.93874,5.00,1.000,"[210.3, 141.3, 89.8, 78.4, 70.9, 66.6, 67.5, 7..."
5,Choline,9,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0...",2.024615e+07,90375.769385,0.96936,2.50,0.010,"[160.3, 123.7, 91.7, 86.4, 81.9, 79.3, 81.2, 9..."
6,Choline,8,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0]",2.385758e+07,20581.779316,0.99425,1.00,0.005,"[117.0, 109.5, 94.9, 95.0, 93.1, 92.1, 95.0, 1..."
7,Choline,7,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5]",2.551660e+07,-6663.770464,0.99961,0.50,0.005,"[101.2, 105.0, 97.1, 99.4, 98.5, 98.1, 101.4]"
8,Choline,6,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25]",2.601949e+07,-13776.523335,0.99998,0.25,0.005,"[97.3, 104.1, 97.9, 100.8, 100.2, 99.9]"
9,Choline,5,"[0.005, 0.01, 0.025, 0.05, 0.1]",2.595996e+07,-13055.322296,0.99991,0.10,0.005,"[97.7, 104.1, 97.8, 100.6, 100.0]"


In [73]:
# Function to filter results
def filter_results(df, r_squared = 0.99):
    # Group by 'molecule' column
    grouped_df = df.groupby('molecule')
    # Filter rows where 'loq' has the lowest value within each group
    lowest_loq_df = grouped_df.apply(lambda x: x[x['loq'] == x['loq'].min()]).reset_index(drop=True)
    # Filter rows where 'r_squared' is larger than defined (default: 0.99) and save results to csv file
    filtered_df = lowest_loq_df[lowest_loq_df['r_squared'] > r_squared]
    filtered_df.to_csv('02_filtered_results.csv', index=False)
    # Keep only the first entry per 'molecule' and save results to csv file
    first_entry_df = filtered_df.groupby('molecule').first().reset_index()
    first_entry_df.to_csv('03_first_entry_results.csv', index=False)
    # Identify and save molecules that failed the filter
    failed_molecules = set(df['molecule']) - set(filtered_df['molecule'])
    failed_df = df[df['molecule'].isin(failed_molecules)]
    failed_df.to_csv('04_failed_molecules.csv', index=False)
    
    return first_entry_df

In [74]:
# Apply filtering process
first_entry_df = filter_results(df)
first_entry_df.head()

,molecule,num_levels,levels,slope,intercept,r_squared,upper_level,loq,accuracies
0,Choline,8,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0]",2.385758e+07,20581.779316,0.99425,1.0,0.005,"[117.0, 109.5, 94.9, 95.0, 93.1, 92.1, 95.0, 1..."
1,Glu_neg,8,"[0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1.0]",2.539873e+05,1800.461027,0.99098,1.0,0.025,"[85.3, 222.0, 85.4, 96.1, 88.6, 86.9, 96.9, 10..."
2,Glu_pos,8,"[0.05, 0.1, 0.25, 0.5, 1.0, 2.5, 5.0, 10.0]",1.158027e+05,-1462.583745,0.99953,10.0,0.050,"[80.2, 99.2, 105.2, 109.7, 108.2, 98.3, 101.7,..."
